This notebook contains an analysis workflow for refining textures in diffraction spectra using MAUD (http://maud.radiographema.eu/)

## Import packages

In [1]:
from decimal import Decimal
import pathlib

from tqdm.notebook import tqdm
import numpy as np

## Notes on preparing the .par file

Before beginning the analysis a template .par file must be set up in MAUD.

## Preparing the DAWN spectra

In DAWN the data must be caked with the x-axis chosen as pixel number (pixels) and saved as a .dat file. 

It is not possible to adjust data using x-axis as the two-theta angle (degrees).

## User parameters for preparing analysis files

In [ ]:
# user inputs (in mm)
detector_distance = 1194.8865
pixel_size = 0.296

# Test number is put in the name of the input file
test_number = 65

# The formatting of the name of the input files
file_prefix = 'adc_{:03d}_TI64_NDload_900C_15mms'.format(test_number)
dir_name = 'data/' + file_prefix + '_ascii/'

# This specifies the angle of the first cake in the data file.
start_angle = 0

# Number and spacing of files to read
start = 0
end = 10
step = 1

# Where the .esg files are written
output_folder = pathlib.Path("analysis")

## Loop through multiple input files to output .esg data files for MAUD

Normally we have many analysis files spaced in time. We can read these in a loop and produce one .esg data files for each input file.

In this case the output files are written into a folder called 'analysis' and saved in the format MAUD_065_00001.esg where 065 is experiment number and 00001 is test number.

In [ ]:
# loop through the data using the image number of the files
for image_number in tqdm(range(start, end + 1, step)):
    
    # caked synchrotron data from DAWN
    input_file = dir_name + file_prefix + '_{:05d}.dat'.format(image_number)
    input_data = np.loadtxt(input_file)
    # Read the pixel spacing from the data file and multiply by pixel size to get pixel positions in mm
    pixel_list = input_data[:, 0] * pixel_size
    
    cake_spectrum = input_data[:, 1:]
    number_of_cakes = cake_spectrum.shape[1]
    cake_width = 360 / number_of_cakes
    
    # Name and open the output MAUD data file
    output_folder.mkdir(exist_ok=True)
    output_path = output_folder / pathlib.Path('MAUD_{:03d}_{:05d}.esg'.format(test_number,image_number))
    
    with open(output_path, 'w') as output_file:

        # write metadata for the top of the file
        output_file.write('_pd_block_id noTitle|#0\n\
        _diffrn_detector Image Plate\n\
        _diffrn_detector_type Image Plate\n\
        _pd_meas_step_count_time ?\n\
        _diffrn_measurement_method ?\n\
        _diffrn_measurement_distance_unit mm\n\
        _pd_instr_dist_spec/detc {:.4f}\n'.format(detector_distance))
        output_file.write('_diffrn_radiation_wavelength ?\n\
        _diffrn_source_target ?\n\
        _diffrn_source_power ?\n\
        _diffrn_source_current ?\n\
        _pd_meas_angle_omega 0.0\n\
        _pd_meas_angle_chi 0.0\n\
        _pd_meas_angle_phi 0.0\n\
        _riet_par_spec_displac_x 0\n\
        _riet_par_spec_displac_y 0\n\
        _riet_par_spec_displac_z 0\n\
        _riet_meas_datafile_calibrated false\n\
        _pd_meas_angle_eta {:.1f}\n\n'.format(start_angle))
        output_file.write('loop_\n\
        _pd_proc_2theta_corrected\n\
        _pd_meas_intensity_total\n')

        # write the first cake intensity data
        for i in range(len(pixel_list)-1,-1,-1):
            output_file.write('{:.3f}\t{:.8f}\n'.format(pixel_list[i], cake_spectrum[i][0]))

        # write all the other cake data with additional info
        for cake_number in range(1, number_of_cakes, 1):    
            output_file.write('\n_pd_block_id noTitle|#{:.0f}\n\n'.format(cake_number))
            cake_angle = start_angle + cake_number * cake_width
            output_file.write('_pd_meas_angle_eta {:.1f}\n\n'.format(cake_angle))
            output_file.write('loop_\n\
            _pd_proc_2theta_corrected\n\
            _pd_meas_intensity_total\n')

            # write cake intensity data for each cake
            for i in range(len(pixel_list)-1, -1, -1):
                output_file.write('{:.3f}\t{:.8f}\n'.format(pixel_list[i], cake_spectrum[i][cake_number]))
                
print(f"{int((end + 1 - start) / step)} .esg data files written to '{output_folder}' folder.")

## Output many .par analysis files for MAUD batch mode analysis

Write out a number of .par analysis files to be refined using the MAUD batch mode. 


A starting MAUD par analysis file is needed to swap the data into. This file will have been manually refined in MAUD and be setup to refine in batch mode. It is assumed that this start MAUD par file is named in the format start_065.par, where 065 is the experiment number.


The output files are written into an 'analysis' folder and saved in the format MAUD_065_00001.par where 065 is experiment number and 00001 is test number. This analysis folder will then be used in the subsequent batch mode analysis.

In [ ]:
# path to the start_***.par file
par_dir_name = 'data/par/'

# Name of the data (.esg files) to be replaced in the MAUD start_***.par file
previous_esg_filename = 'MAUD_{:03d}_00000'.format(test_number)

# MAUD file to swap the data into
template_par = par_dir_name + 'start_{:03d}.par'.format(test_number)


for image_number in tqdm(range(start, end + 1, step)):
    
    # caked synchrotron data from DAWN
    input_file = dir_name + file_prefix + '_{:05d}.dat'.format(image_number)
    input_data = np.loadtxt(input_file)
    # Read the pixel spacing from the data file and multiply by pixel size to get pixel positions in mm
    pixel_list = input_data[:, 0] * pixel_size
    
    cake_spectrum = input_data[:, 1:]
    number_of_cakes = cake_spectrum.shape[1]
    cake_width = 360 / number_of_cakes
    
   
    # output MAUD analysis file
    output_par_path = output_folder / pathlib.Path(f'MAUD_{test_number:03d}_{image_number:05d}.par')
    
    with open(template_par, 'r') as template_par_file, open(output_par_path, 'w') as new_par_file:
        cake_number = 0
        line = template_par_file.readline()
            
        while line:
            # rename the datafiles in the .par file
            if previous_esg_filename in line:
                new_esg_filename = 'MAUD_{:03d}_{:05d}'.format(test_number, image_number)
                new_line = line.replace(previous_esg_filename, new_esg_filename)
                new_par_file.write(new_line)
                
            # replace the data when this line is found
            elif '_pd_meas_position _pd_meas_intensity_total _pd_meas_intensity_sigma' in line:
                new_par_file.write(line)
                
                # write the new data in place
                for j in range(len(pixel_list)-1, -1, -1):
                    template_par_file.readline()
                    new_par_file.write('{:.3f}\t{:.8f}\t1.0\n'.format(pixel_list[j], cake_spectrum[j][cake_number]))
                cake_number += 1
            
            # write the rest of the file
            else:
                new_par_file.write(line)
            
            line = template_par_file.readline()
            
print(f"Written {int((end + 1 - start) / step)} files to '{output_folder}' folder.")

## Write an instruction file for the batch mode

To run the analysis in batch mode, an instruction file is required to tell MAUD exaclty what analysis to do. This section writes the .ins file.

The number 13 corresponds to the refinement analysis in MAUD. The number of iterations sets how many times to run the refinement on each sample.

In [ ]:
# user inputs
number_of_iterations = 7

batch_path = pathlib.Path('MAUD_batch_065.ins')
with batch_path.open(mode='w') as output_file:

    output_file.write("loop_\n\
    _riet_analysis_file\n\
    _riet_analysis_iteration_number\n\
    _riet_analysis_wizard_index\n\
    _riet_analysis_fileToSave\n\
    _riet_append_simple_result_to\n\n")
    # note, _riet_meas_datafile_name\n\ removed from above for swapping the files.
    # note, also _riet_append_result_to\n\ but this is only the Rwp value

    for file_number in range (start, end + 1, step):
        output_file.write(f"'/analysis/MAUD_{test_number:03d}_{file_number:05d}.par' "
                          f"{number_of_iterations} 13 '/analysis/MAUD_{test_number:03d}_"
                          f"{file_number:05d}_refined.par' '/analysis/batch_results.txt\'\n")
print(".ins file written.")

## Running MAUD batch mode

Place the instruction .ins file and a folder for the analysis (containing the newly created .par analysis files) in a sensible location, such as; 


**/Users/mbcx9cd4/Documents/maud/batch/**


_Note, be careful with using a dropbox folder such as /Users/mbcx9cd4/Dropbox\ \(Research\ Group\)/Lightform\ Postdoc/MAUD_


In the terminal navigate to the hidden files within the maud.app using the command; 


**cd /Applications/Maud.app/Contents/Resources/Java**


_Note, you can view these files by right clicking on show package contents._


Then, run the command to start the batch mode;


**java -mx2048M -cp Maud.jar:miscLib.jar:jgaec.jar:ij.jar com.radiographema.MaudText -f /Users/mbcx9cd4/Documents/maud/batch/MAUD_batch_065_mac.ins**


A pop-up window will then appear, navigate to the .ins instruction file and click open. The batch mode will then run, showing the refinements in the terminal.

## Plotting refinement results

In [ ]:
results_file = 'analysis/batch_results.txt'
results = np.loadtxt(result_file, usecols = (0), skiprows=1)
print('number of pixels =', len(pixel_number))